In [1]:
import numpy as np
import qiskit as qk

# Generate Random Vector $|\varphi>$ on 4 Qubits
The quantum state of 4 qubits can be specified by a vector of 2^4 amplitudes.

For simplicity, I am considering only real values for amplitudes.

In [47]:
phi = 2*np.random.rand(16)-1 # 16 random values between -1 and 1
phi = phi/np.linalg.norm(phi) # Normalizing
print("Quantum State of 4 qubits =", phi) # valid quantum state of 4 qubits
print("Total Probability =", np.sum(phi**2)) # probability sums to one

Quantum State of 4 qubits = [ 0.24153199 -0.10654712  0.3602784   0.16770457 -0.24871177  0.00341616
 -0.40233047  0.0230149   0.06631768  0.36737779 -0.08719295  0.4040572
  0.19380053  0.3287001  -0.08563866  0.29146228]
Total Probability = 1.0


# Implement Quantum Circuit
Take the initial quantum state (phi), number of layers (L), and variational parameters (thetas) as input to get the output state

In [48]:
class QuantumCircuitSim:
    
    # phi: random generated vector on 4 qubits so has 2^4 values denoting the quantum state
    # L: number of layers
    # thetas: variational parameters of all blocks in all layers as a single vector, contains L*2*4 values
    def __init__(self, phi, L, thetas):
        self.q = qk.QuantumRegister(4) # qubits
        self.qc = qk.QuantumCircuit(self.q) # circuit
        self.qc.initialize(phi, [self.q[0], self.q[1], self.q[2], self.q[3]])
        
        for i in range(L):
            start_index = i*2*4
            theta1 = thetas[start_index:start_index+4]
            theta2 = thetas[start_index+4:start_index+8]
            self.layer(theta1, theta2)
            
            
    # theta1: variational paramters of odd block in the layer
    # theta2: variational paramters of even block in the layer
    def layer(self, theta1, theta2):
        # odd block
        self.qc.rx(theta1[0], self.q[0])
        self.qc.rx(theta1[1], self.q[1])
        self.qc.rx(theta1[2], self.q[2])
        self.qc.rx(theta1[3], self.q[3])
        
        # even block
        self.qc.rz(theta2[0], self.q[0])
        self.qc.rz(theta2[1], self.q[1])
        self.qc.rz(theta2[2], self.q[2])
        self.qc.rz(theta2[3], self.q[3])
        
        self.qc.cz(self.q[0], self.q[1])
        self.qc.cz(self.q[0], self.q[2])
        self.qc.cz(self.q[0], self.q[3])
        
        self.qc.cz(self.q[1], self.q[2])
        self.qc.cz(self.q[1], self.q[3])
        
        self.qc.cz(self.q[2], self.q[3])
        
        return
    
    def output(self):
        backend = qk.BasicAer.get_backend('statevector_simulator')
        job = qk.execute(self.qc, backend)
        qc_state = job.result().get_statevector(self.qc)
        return qc_state

# Calculate Distance

In [56]:
def distance(phi, L, thetas):
    qcs = QuantumCircuitSim(phi, L, thetas)
    output_state = qcs.output()
    return np.linalg.norm(output_state-phi)**2

# Example
Number of layers (L) = 1

In [62]:
L = 1
thetas = np.random.rand(L*2*4)*2*np.pi # L*2*4 values for variational parameters
print("phi =", phi)
print("Thetas =", thetas)
qcs = QuantumCircuitSim(phi, L, thetas)
print("Output State =", qcs.output())
print("Distance =", distance(phi, L, thetas))
qcs.qc.draw()

phi = [ 0.24153199 -0.10654712  0.3602784   0.16770457 -0.24871177  0.00341616
 -0.40233047  0.0230149   0.06631768  0.36737779 -0.08719295  0.4040572
  0.19380053  0.3287001  -0.08563866  0.29146228]
Thetas = [1.00061443 4.23793329 2.47599028 1.10542537 3.95875895 0.04614394
 2.20519789 0.68349002]
Output State = [ 0.07438203+0.37577821j -0.0930433 -0.00822295j  0.04793098+0.21466487j
  0.0255898 -0.16038593j  0.20358474-0.02600755j  0.21950236+0.02593232j
 -0.1321992 +0.054292j    0.34815993-0.12477085j -0.01115215-0.09920891j
  0.27421089+0.01211053j -0.17367535+0.10694775j  0.33070765-0.06112639j
 -0.34960583+0.16947776j -0.14459156+0.18583101j -0.12259171+0.08902453j
  0.19204485-0.12393587j]
Distance = 1.478570848668026


»
q733_0: »
        »
q733_1: »
        »
q733_2: »
        »
q733_3: »
        »
«        ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐»
«q733_0: ┤0                                                                                                                                                     ├»
«        │                                                                                                                                                      │»
«q733_1: ┤1                                                                                                                                                     ├»
«        │  initialize(0.24153,-0.10655,0.36028,0.1677,-0.24871,0.0034162,-0.40233,0.023015,0.066318,0.36738,-0.087193,0.40406,0.1938,0.3287,-0.085639,0.29146) │»
«q733_2: ┤2                                                                                                                                                     ├»
«        │                                                                                                                                                      │»
«q733_3: ┤3                                                                                                                                                     ├»
«        └──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘»
«        ┌────────────┐ ┌────────────┐                   
«q733_0: ┤ RX(1.0006) ├─┤ RZ(3.9588) ├──■──■──■──────────
«        ├────────────┤┌┴────────────┴┐ │  │  │          
«q733_1: ┤ RX(4.2379) ├┤ RZ(0.046144) ├─■──┼──┼──■──■────
«        ├───────────┬┘└┬────────────┬┘    │  │  │  │    
«q733_2: ┤ RX(2.476) ├──┤ RZ(2.2052) ├─────■──┼──■──┼──■─
«        ├───────────┴┐┌┴────────────┤        │     │  │ 
«q733_3: ┤ RX(1.1054) ├┤ RZ(0.68349) ├────────■─────■──■─
«        └────────────┘└─────────────┘